In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
from models.Unet_Resnet import Unet_Resnet50

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
model_dir = os.path.join(os.getcwd(), 'config')
unet_model = Unet_Resnet50(model_dir = model_dir,
                           visible_gpu = 0,
                           config_filepath=os.path.join(model_dir, 'Unet_Resnet50-20191017T0347-config.yml'),
                           for_prediction=True, 
                           tile_size = [512,512]
                          )
unet_model.initialize_model()
unet_model.load_weights(os.path.join(model_dir, 'Unet_Resnet50-20191017T0347_0155.h5'))


Loaded config file from C:\Users\mbigwj\Documents\Cell-Chirality-Analysis-main\Actin_segmentation\config\Unet_Resnet50-20191017T0347-config.yml
Updating model_dir to C:\Users\mbigwj\Documents\Cell-Chirality-Analysis-main\Actin_segmentation\config


Unet_Resnet50 2021-01-20 17:39:24.738194





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Predicting using single GPU or CPU..
Using 1 - Dice + BCE loss

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loaded weights from: C:\Users\mbigwj\Documents\Cell-Chirality-Analysis-main\Actin_segmentation\config\Unet_Resnet50-20191017T0347_0155.h5


In [6]:
root = '../Test_data'
os.listdir(root)


['Control', 'siINF2']

In [7]:
#  Check incomplete segmentation
warnings.simplefilter(action='ignore', category=UserWarning)
exceptions = []

actin_folders = sorted([i for i, _, _ in os.walk(root) if i.endswith('actin')])
for folder in actin_folders:
    maskdir = os.path.join(folder, 'Masks/Unet_Resnet50-20191017T0347_0155')
    try:  
        print(folder)
        temp = unet_model.predict_images(folder)
    except Exception as e:
        exceptions.append((e, folder))

if exceptions:
    for exception, folder in exceptions:
        print(exception, folder)

print("Done")
       

../Test_data\Control\actin
1/1 [==============================] - 7s 7s/step
../Test_data\siINF2\actin
1/1 [==============================] - 7s 7s/step
Done


In [10]:
unet_model.end_training()